In [105]:
import pandas as pd
import numpy as np 
import re

from ast import literal_eval

In [209]:
deck = pd.read_csv("parser_output/WLL.csv")

In [210]:
card_groups = deck.groupby("card_id")

In [211]:
card_groups.get_group(10)

,name,cmc,text,mana_cost,card_id,is_dfc,color_identity,board,power,toughness,supertype,type,subtype
10,Goblin Engineer,2.0,"When Goblin Engineer enters the battlefield, y...",{1}{R},10,False,['R'],mainboard,1,2,[],['Creature'],"['Goblin', 'Artificer']"


In [212]:
karsten = pd.read_csv("data/karsten_tables.csv", sep="\t")

In [213]:
karsten

,Lands,C,P(oncurve),1C,CC,P(oncurve).1,2C,1CC,CCC,P(oncurve).2,...,7C,6CC,5CCC,4CCCC,3CCCCC,2CCCCCC,1CCCCCCC,CCCCCCCC,P(oncurve).7,Avg.handsize:
0,24,15,96.67%,15,22,95.82%,13,19,23,71.34%,...,7,10,13,16,19,21,23,24,1.40%,6.72cards
1,25,15,96.41%,15,23,96.09%,13,20,24,73.56%,...,7,11,14,17,20,22,24,25,1.83%,6.75cards
2,26,16,96.88%,16,23,96.20%,14,20,25,75.60%,...,7,11,15,18,20,23,25,26,2.36%,6.79cards
3,27,16,96.65%,16,24,96.39%,14,21,26,77.50%,...,8,12,15,18,21,24,26,27,2.98%,6.82cards
4,28,17,97.05%,16,25,96.52%,14,22,27,79.25%,...,8,12,16,19,22,24,27,28,3.71%,6.84cards
5,29,17,96.84%,17,25,96.58%,15,22,28,80.81%,...,8,13,16,20,23,25,28,29,4.56%,6.86cards
6,30,18,97.20%,17,26,96.70%,15,23,29,82.30%,...,8,13,17,20,23,26,28,30,5.52%,6.88cards
7,31,18,97.01%,18,27,97.09%,16,24,30,83.83%,...,9,13,18,21,24,27,29,31,6.63%,6.90cards
8,32,18,96.84%,18,27,96.81%,16,24,30,84.68%,...,9,14,18,22,25,28,30,32,7.87%,6.91cards
9,33,19,97.16%,18,28,96.90%,16,25,31,85.87%,...,9,14,19,22,26,29,31,33,9.23%,6.93cards


In [215]:
literal_eval(deck[deck["board"]=="commander"]['color_identity'].values[0])

['B', 'W']

In [216]:
def find_identity(deck):
    commanders = deck[deck["board"]=="commander"]
    identity = []
    for i in range(len(commanders)):
        identity += literal_eval(commanders['color_identity'].iloc[i])
    return np.unique(identity)

In [217]:
def build_dict(identity):
    dictionary = {}
    for i in identity:
        dictionary[i] = {}
    dictionary["C"] = {}
    return dictionary

In [218]:
i_dict = build_dict(find_identity(deck))

In [219]:
class meta_deck:
    
    def __init__(self, deck, table):
        self.deck = deck
        self.id = find_identity(deck)
        self.spells_casting_costs = build_dict(find_identity(deck))
        self.cast_table = table
        
    
    def fill_costs(self):
        for i in range(len(self.deck)):
            entry = self.deck.iloc[i]
            
            # Checking if the entry has a mana_cost
            if type(entry["mana_cost"]) is str:
                m = entry["mana_cost"]
                
                
                # Processing the mana cost into a list
                m_ = []
                for s in re.findall('\{.*?\}',m):
                    try:
                        temp = int(s[s.find("{")+1:s.find("}")])
                        m_.append(temp)
                    except ValueError:  
                        m_.append(s[s.find("{")+1:s.find("}")])
                
                flag_colorless = True
                for i in self.id:
                    if i in m_:
                        flag_colorless = False
                        count_colorless = 0
                        count_color = 0
                        for cost in m_:
                            if type(cost) is int:
                                count_colorless+=cost
                            else:
                                if i in cost:
                                    count_color+=1
                                else:
                                    count_colorless+=1
                        
                        if count_colorless > 0:
                            key = str(count_colorless) + "C"*count_color
                        else:
                            key = "C"*count_color
                        
                        try:
                            self.spells_casting_costs[i][key].append(entry["name"])
                        except KeyError:
                            self.spells_casting_costs[i][key]=[]
                            self.spells_casting_costs[i][key].append(entry["name"])
                            
                if flag_colorless:
                    key = str(int(entry["cmc"]))
                    try:
                        self.spells_casting_costs["C"][key].append(entry["name"])
                    except KeyError:
                        self.spells_casting_costs["C"][key]=[]
                        self.spells_casting_costs["C"][key].append(entry["name"])                   
                
                        


In [220]:
meta = meta_deck(deck, karsten)
meta.fill_costs()

In [221]:
spell_dict = meta.spells_casting_costs

In [225]:
print("Nombre de sources B pour caster", spell_dict["B"]["2C"], " on curve :")
print(karsten[karsten["Lands"]==38]["2C"].iloc[0])

Nombre de sources B pour caster ['Toxic Deluge', 'Void Rend']  on curve :
18


In [223]:
spell_dict

{'B': {'1C': ["Wernog, Rider's Chaplain",
   'Demonic Tutor',
   'Wishclaw Talisman',
   'Tainted Pact',
   'Shred Memory',
   'Dimir Infiltrator',
   'Baleful Strix',
   'Deadly Dispute'],
  'C': ['Rain of Filth',
   'Duress',
   'Inquisition of Kozilek',
   'Thoughtseize',
   'Fatal Push',
   'Dark Ritual'],
  '2C': ['Toxic Deluge', 'Void Rend'],
  '3C': ['Deadly Rollick']},
 'R': {'1C': ['Bjorna, Nightfall Alchemist',
   'Goblin Engineer',
   'Underworld Breach',
   'Rip Apart',
   'Expressive Iteration'],
  'C': ['Lightning Bolt']},
 'U': {'1C': ['Bjorna, Nightfall Alchemist',
   'Mana Leak',
   'Evasive Action',
   'Brain Freeze',
   'Remand',
   'Lose Focus',
   'Dimir Infiltrator',
   'Expressive Iteration',
   'Dress Down',
   'Memory Lapse',
   'Baleful Strix',
   "Tale's End"],
  '3C': ['Supreme Verdict'],
  'C': ['Brainstorm',
   'Preordain',
   'Ponder',
   'Chain of Vapor',
   'Hard Evidence'],
  '2C': ['Intuition', 'Solve the Equation', 'Void Rend'],
  'CC': ['Muddle the 